In [1]:
import albumentations as A
import os
import random
import config
import sys
import cv2
import numpy as np
import shutil


sys.path.append('../util')
from DatasetAugmentation import *


#Insira o nome do root do dataset original
original_dataset_path = './dataset/fuseg-original-dataset'
output_base = config.dataset_path

/home/alan/miniconda3/envs/pytorch5070/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Removing images without masks from dataset

In [2]:


def mover_mascaras_pretas(
    dir_mascaras, 
    dir_imagens, 
    dir_destino, 
    extensoes=(".png", ".jpg", ".jpeg", ".tif")
):
    os.makedirs(dir_destino, exist_ok=True)
    mascaras_pretas = []

    for root, _, files in os.walk(dir_mascaras):
        for file in files:
            if file.lower().endswith(extensoes):
                caminho_mask = os.path.join(root, file)
                img = cv2.imread(caminho_mask, cv2.IMREAD_UNCHANGED)

                if img is None:
                    print(f"[AVISO] Não foi possível ler {caminho_mask}")
                    continue

                # Converte para escala de cinza se tiver múltiplos canais
                if len(img.shape) == 3:
                    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

                # Verifica se todos os pixels são 0
                if np.all(img == 0):
                    nome_base, _ = os.path.splitext(file)
                    mascaras_pretas.append(nome_base)

                    # Caminho da imagem correspondente
                    caminho_img = None
                    for ext in extensoes:
                        tentativa = os.path.join(dir_imagens, nome_base + ext)
                        if os.path.exists(tentativa):
                            caminho_img = tentativa
                            break

                    # Mover máscara
                    novo_nome_mask = f"{nome_base}_mask.png"
                    destino_mask = os.path.join(dir_destino, novo_nome_mask)
                    shutil.move(caminho_mask, destino_mask)

                    # Mover imagem, se existir
                    if caminho_img and os.path.exists(caminho_img):
                        destino_img = os.path.join(dir_destino, os.path.basename(caminho_img))
                        shutil.move(caminho_img, destino_img)
                    else:
                        print(f"[AVISO] Imagem correspondente não encontrada para {file}")

                    print(f"[OK] Movido: {file} e imagem correspondente")

    print(f"\nTotal de máscaras pretas movidas: {len(mascaras_pretas)}")
    return mascaras_pretas


# ======== Exemplo de uso ========

# Caminho das máscaras originais
dir_mascaras = f"{original_dataset_path}/train/labels"
# Caminho das imagens originais
dir_imagens = f"{original_dataset_path}/train/images"
# Caminho de destino para onde serão movidas
dir_destino = f"{original_dataset_path}/train-fix/"

mover_mascaras_pretas(dir_mascaras, dir_imagens, dir_destino)


[OK] Movido: 0092.png e imagem correspondente
[OK] Movido: 0131.png e imagem correspondente
[OK] Movido: 0223.png e imagem correspondente
[OK] Movido: 0281.png e imagem correspondente
[OK] Movido: 0335.png e imagem correspondente
[OK] Movido: 0361.png e imagem correspondente
[OK] Movido: 0457.png e imagem correspondente
[OK] Movido: 0502.png e imagem correspondente
[OK] Movido: 0564.png e imagem correspondente
[OK] Movido: 0600.png e imagem correspondente
[OK] Movido: 0701.png e imagem correspondente
[OK] Movido: 0725.png e imagem correspondente
[OK] Movido: 0736.png e imagem correspondente
[OK] Movido: 0778.png e imagem correspondente
[OK] Movido: 0832.png e imagem correspondente
[OK] Movido: 0899.png e imagem correspondente
[OK] Movido: 0912.png e imagem correspondente
[OK] Movido: 0931.png e imagem correspondente
[OK] Movido: 0950.png e imagem correspondente

Total de máscaras pretas movidas: 19


['0092',
 '0131',
 '0223',
 '0281',
 '0335',
 '0361',
 '0457',
 '0502',
 '0564',
 '0600',
 '0701',
 '0725',
 '0736',
 '0778',
 '0832',
 '0899',
 '0912',
 '0931',
 '0950']

### Moving 100 images from validation to test

In [7]:
from_img_dir = os.path.join(original_dataset_path, "validation", "images")
from_lbl_dir = os.path.join(original_dataset_path, "validation", "labels")
to_img_dir = os.path.join(original_dataset_path, "test", "images")
to_lbl_dir = os.path.join(original_dataset_path, "test", "labels")

os.makedirs(to_img_dir, exist_ok=True)
os.makedirs(to_lbl_dir, exist_ok=True)

images = os.listdir(from_img_dir)
selected = random.sample(images, 100)

for img in selected:
    shutil.move(os.path.join(from_img_dir, img), os.path.join(to_img_dir, img))
    shutil.move(os.path.join(from_lbl_dir, img), os.path.join(to_lbl_dir, img))

### Augmentation

In [8]:

# -----------------------------
# Parâmetros
# -----------------------------
N = 10  # número de aumentações
num_to_valid = 0    # número de imagens a mover do train para valid
num_to_test  = 0    # número de imagens a mover do train para test



target_size  = (224, 224)
random.seed(42)

# -----------------------------
# Caminhos de entrada e saída
# -----------------------------
orig_train_img_dir  = os.path.join(original_dataset_path, 'train/images')
orig_train_mask_dir = os.path.join(original_dataset_path, 'train/labels')
orig_valid_img_dir  = os.path.join(original_dataset_path, 'validation/images')
orig_valid_mask_dir = os.path.join(original_dataset_path, 'validation/labels')
orig_test_img_dir   = os.path.join(original_dataset_path, 'test/images')
orig_test_mask_dir  = os.path.join(original_dataset_path, 'test/labels')


output_dirs = {
    'train_images': os.path.join(output_base, 'images/train'),
    'train_labels': os.path.join(output_base, 'labels/train'),
    'valid_images': os.path.join(output_base, 'images/valid'),
    'valid_labels': os.path.join(output_base, 'labels/valid'),
    'test_images':  os.path.join(output_base, 'images/test'),
    'test_labels':  os.path.join(output_base, 'labels/test'),
}

transforms = A.Compose([
    A.Resize(*target_size, interpolation=cv2.INTER_NEAREST),
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.RandomRotate90(p=0.5),
    A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.1, rotate_limit=30, p=0.7, border_mode=cv2.BORDER_REFLECT),
    A.RandomBrightnessContrast(p=0.5),
    A.ElasticTransform(p=0.2),
    A.GaussianBlur(p=0.3),
    A.GridDistortion(p=0.2),
])

augment_dataset(N, num_to_valid, num_to_test,
                    orig_train_img_dir, orig_train_mask_dir,
                    orig_valid_img_dir, orig_valid_mask_dir,
                    orig_test_img_dir, orig_test_mask_dir,
                    output_base,
                    transforms,
                    )

Imagens totais no dataset original: 791
→ Treino: 791
→ Validação (do treino): 0
→ Teste (do treino): 0

Com N=10, total de imagens geradas no treino será: 8701


/home/alan/miniconda3/envs/pytorch5070/lib/python3.10/site-packages/albumentations/core/validation.py:114: UserWarning: ShiftScaleRotate is a special case of Affine transform. Please use Affine transform instead.
  original_init(self, **validated_kwargs)
Copiando test: 100%|██████████| 300/300 [00:00<00:00, 555.13it/s]


→ 100 imagens copiadas da pasta valid original.
→ 100 imagens copiadas da pasta test original.


Aumentando imagens de treino: 100%|██████████| 791/791 [01:23<00:00,  9.43it/s]


Resumo final:
train_images: 8701 arquivos
train_labels: 8701 arquivos
valid_images: 100 arquivos
valid_labels: 100 arquivos
test_images: 100 arquivos
test_labels: 100 arquivos
